In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numpy.random as rd
from scipy import stats
import copy
import os

from pyperplan import planner as pl
from pyperplan import search as sc
from pyperplan.planner import (
    find_domain,
    HEURISTICS,
    search_plan,
    SEARCHES,
    validate_solution,
    write_solution,
)
from pyperplan.task import Operator, Task
from pyperplan.heuristics.heuristic_base import Heuristic
from pyperplan.search.a_star import greedy_best_first_search as gbfs
from typing import Set

# Planning Graph

In [3]:
# define the planning graph
class RelaxedGraph(object):
    
    def __init__(self): # initialise the Graph
        self.num_of_levels: int = 0
        self.act = {0: None}
        self.prop = {}
        self.fixed_point = False
        
            

class RelaxedPlanningGraph(object):
    
    def __init__(self, domain_file: str, problem_file: str):
        # initialise the relaxed planning graph
        self.task = pl._ground(pl._parse(domain_file, problem_file))
        self.graph = None
        self.plan = None
        self.hff = -1 # not yet generated
        self.dom = domain_file
        self.prob = problem_file
        self.success = False # whether successfully generated or not
        
    def create(self, max_level, state = None):
        # create the planning graph with a initial state specified
        # return the level of relaxed graph generated, -1 if reached fixed point, -2 if reached max depth
        self.graph = RelaxedGraph()
        if state is not None:
            self.graph.prop = {0: set(state)}
        else:
            self.graph.prop = {0: set(self.task.initial_state)}
        goal_set = self.task.goals
 
        for level in range(max_level+1):
            current_props = self.graph.prop[level]
            # if the goal has been satisfied
            if Task.goal_reached(self.task, current_props):
                self.success = True
                return level
            
            # else expand the relaxed graph
            self.graph.act[level+1] = set([op for op in self.task.operators if op.applicable(current_props)])
            
            next_props = current_props.copy()
            for op in self.graph.act[level+1]:
                next_props = next_props | op.add_effects
            
            if len(current_props) == len(next_props):
                return -1 # reached fixed point before finding the goal
            self.graph.prop[level+1] = next_props
        
        return -2 #reached max depth
        
    def hff_plan(self):
        # generate a relaxed plan with hff 
        # first return error if graph not successfully generated
        # return hff, -1 if failed
        if self.graph is None:
            print("Graph not yet generated")
            return -1 # graph not yet created
        if not self.success:
            print("Invalid graph")
            return -1 # graph does not reach goal state
        
        # otherwise start backtrace
        # setup g_k
        current_goal = set(self.task.goals.copy())
        k = len(self.graph.act.keys())-1
        self.plan = {}
        for i in range(k, 0, -1):
            act_set = set()
            # select the minimum set of actions that r-satisfied current goal
            for a in self.graph.act[i]:
                for eff in a.add_effects:
                    if eff in current_goal:
                        current_goal.remove(eff)
                        act_set.add(a)
            # update the current goal to be the goals for previous layer
            for a in act_set:
                current_goal.update(a.preconditions)
                
            # update the final plan
            self.plan[i] = list(act_set)

        
        if current_goal.issubset(self.graph.prop[0]):
            count = 0
            for layer in self.plan.values():
                count+=len(layer)
            self.hff = count
            return self.hff
        else:
            print("something went wrong during planning")
            return -1
            
            
        

In [26]:
rpl = RelaxedPlanningGraph("datasets/domain/blocks/domain.pddl", 
                        "datasets/domain/blocks/blocks/blocks4/task01.pddl")
plan = read_plan("datasets/domain/blocks/plans/blocks4-task01_1800.out")
rpl.create(999)

graph = rpl.graph

ini = graph.prop[0].copy()

for a in rpl.task.operators:
    if a.applicable(ini):
        print(f'action:{a.name}, state:', a.apply(ini.copy()))

op1 = find_operator(plan[0], rpl.task.operators)

ini = op1.apply(ini)
print()
for a in rpl.task.operators:
    if a.applicable(ini):
        print(f'action:{a.name}, state:', a.apply(ini.copy()))
    
# graph.prop

action:(pick-up b3), state: {'(on b1 b2)', '(on-table b2)', '(on b4 b1)', '(clear b4)', '(holding b3)'}
action:(unstack b4 b1), state: {'(holding b4)', '(clear b3)', '(on b1 b2)', '(on-table b3)', '(on-table b2)', '(clear b1)'}

action:(put-down b4), state: {'(clear b3)', '(on-table b4)', '(on b1 b2)', '(on-table b3)', '(on-table b2)', '(clear b1)', '(clear b4)', '(handempty)'}
action:(stack b4 b3), state: {'(on b1 b2)', '(on-table b3)', '(on-table b2)', '(on b4 b3)', '(clear b1)', '(clear b4)', '(handempty)'}
action:(stack b4 b1), state: {'(clear b3)', '(on b1 b2)', '(on-table b3)', '(on-table b2)', '(on b4 b1)', '(clear b4)', '(handempty)'}


In [4]:
fs = frozenset([6, 7, 8, 9])
s = {1, 2, 3, 4, 5}

y = {1,2,6}

print(s - y) 

{3, 4, 5}


# Feature Vector 

In [5]:
def generate_feature_vec_relaxed(planning_graph, state, max_level):
    """
    Generate the feature vector followed by the paper from the input (problem, state) pair as described by the paper
    Please notice that action and operator are referring to the same type Operator within this function
    
    Inputs
    ----------
    planning_graph: the relaxed planning graph DAG pi
    state: the current state set of facts, it should never be the goal 
    max_level: the maximum layer allowed for ff algorithm to do forward expanding
    
    Outputs
    ----------
    feature_vec: a vector of length n + 2*n**2 + 3 representing the feature generated from the given (problem, state) pair
                 the first n values are single action feature
                 the second 2*n**2 are pairwise action feature
                 the last 3 are original heuristic value, the number of layers in pi and the number of unsatisfied goals
    """
    def get_name(op_name):
        """
        transfer an operator name from format of "(act_name v1 v2...)" to act_name string
        """
        return op_name.split(" ")[0][1:]
    
    
        
    if planning_graph.task.goals <= state:
        return feature_vec # if already the goal
    
    # get action schema and output list
    act_schema = np.array(list(set([get_name(o.name) for o in planning_graph.task.operators]))) # store names of total action schema
    n = len(act_schema)  # length of action schema
    total_len = n+2*n**2+3
    feature_vec = np.zeros(total_len) # return feature vec, first n is linear, second 2*n**2 is pairwise, last 3 is additional feature

    
    # create and get the graph
    idx = planning_graph.create(max_level, state)
    planning_graph.hff_plan()
    graph = planning_graph.graph
    act_layers = list(graph.act.values()) # list of layers generated, ith value is the list of actions connencting i-1 th states to ith states layer
    # extract linear feature
    #-------------------------------------
    # ith value indicate the num of occurance for ith action of act_schema in the entire graph 
    counter = np.zeros(n)
    for act_layer in act_layers:
        if act_layer is not None:
            for a in act_layer:
                counter[act_schema == get_name(a.name)] += 1 
    feature_vec[0:n] = counter
    
    # extract pair-wise feature
    #-------------------------------------
    # each pair a1, a2 is stored in n + [2*(n*a1+a2), 2*(n*a1+a2)+1]
    # e.g. when a1 is 1, a2 is 3, n is 5, store in 5 + [2*(8),  2*(8)+1]
    
    def to_index(n, index_a1, index_a2, adder):
        """
        return corresponding index in the position of the feature vector
        adder is either 0 or 1
        index_a1, index_a2 refer to move index in act_schema
        """
        return n+2*(n*index_a1+index_a2)+adder
    

    def append_to_dict(a, pre, eff_pos):
        """
        add action a into the dicitonary pre and eff_pos
        """
        for p in a.preconditions:
            current = pre.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            pre[p] = current
            
        for p in a.add_effects:
            current = eff_pos.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            eff_pos[p] = current
            
            return pre, eff_pos


    # define dictionary variables for comparison purpose
    # each dictionary maps a fact(proposition) to a list of actions
    pre = {}
    eff_pos = {}
    
    # add pre and eff into the empty dictionary for the first layer
    for a in act_layers[1]:
        pre, eff_pos = append_to_dict(a, pre, eff_pos)
   
    # loop through second to last action layers
    for i in range(2,len(act_layers)): 
        act_layer = act_layers[i]
        
        # update fecture vec for the entire layer based on all previously visited layers
        for a2 in act_layer:
            # count for num of occurances, use set to avoid multiple counts
            s1 = set() # feature 1 where eff a1 and pre a2 has intersections
            s2 = set() # feature 2 where pre a1 and eff a2 has intersections          
            for p in a2.preconditions:
                current = eff_pos.get(p)
                if current is not None:
                    for a1 in current:
                        s1.add(a1) 

            for p in a2.add_effects:
                current = pre.get(p)
                if current is not None:
                    for a1 in current:
                        s2.add(a1)

            # add index to feature_vec based on set generated:
            index_a2 = int(np.where(get_name(a2.name) == act_schema)[0])
            for a1 in s1:
                # update feature 1 for pair (a1, a2)
                index_a1 = int(np.where(get_name(a1.name) == act_schema)[0])
                feature_vec[to_index(n, index_a1, index_a2,0)]+=1

            for a1 in s2:
                # update feature 2 for pair (a1, a2)
                index_a1 = int(np.where(get_name(a1.name) == act_schema)[0])
                feature_vec[to_index(n, index_a1, index_a2,1)]+=1

        # update pre and eff_pos for the entire layer
        for a2 in act_layer:
            pre, eff_pos = append_to_dict(a2, pre, eff_pos)
           
    # extract final features
    #-------------------------------------
    # add heuristic value, number of layers and number of unsatisfied goals
    # number of layers:
    feature_vec[total_len - 3] = len(act_layers)
    # heuristic value hFF: (number of total actions in the plan)
    feature_vec[total_len - 2] = planning_graph.hff
    # unsatisfied goal: (number of propositions within the initial state that's not in the goal)
    ns_goals = 0
    for fact in state:
        if fact not in planning_graph.task.goals:
            ns_goals += 1
    feature_vec[total_len - 1] = ns_goals
    

    return feature_vec

In [18]:
def find_operator(action : str, ops):
    """
    find an operator from the planning graph's ground operator lists
    
    return: the action operator if found
    """
    for op in ops:
        if op.name == action:
            return op
    return None


def read_plan(plan_file_path: str):
    """
    read all the lines from a plan file directory, remove the last line containing cost
    
    return: a list containing the ground truth plan with length equal to total cost
    """
    with open(plan_file_path, "r") as f:

        # Read the lines of the file into a list of strings
        lines = [line.strip() for line in f.readlines()]

    return lines[:-1]

In [7]:
def generate_training_data(domain_file_path, task_file_path, plan_file_path, problem_num : int):
    """
    generate the feature vector matrix X together with a cost vector y from the given input
    
    Inputs
    ----------
    domain_file_path: the input domain file
    task_file_path: the input problem file
    plan_file_path: the input log file that store the optimal plan
    problem_num: the problem index for this domain
    
    Returns
    ----------
    None, None if no plan can be found (plan has cost 0)
    X : array, shape (plan_length-1, n_features)
        The input feature vec of states from initial states all the way towards the second-last state (one state before goal state)
    Y : array, shape (plan_length-1, 2)
        The input cost vector. If it's a 2D array
        The first column is the true cost pi optimal (assume unit cost)
        The second column is the probelm_num representing the index of this problem
    
    """
    # generate plan and get max level
    plan_actions = read_plan(plan_file_path)
    if len(plan_actions) == 0:
        return None, None
    max_level = len(plan_actions)+2
    
    # generate relaxed planning graph
    planning_graph = RelaxedPlanningGraph(domain_file_path, task_file_path)
    
    # define output matrixes
    X = []
    y = []
    
    # loop from the initial state to the second last state
    current_state = planning_graph.task.initial_state
    current_cost = len(plan_actions)
    for i in range(0,len(plan_actions)-1):
        X.append(generate_feature_vec_relaxed(planning_graph, current_state, max_level))
        y.append(current_cost)
        current_action = find_operator(plan_actions[i], planning_graph.task.operators)
        current_state = current_action.apply(current_state)
        current_cost -=1
        
    y = np.c_[y, problem_num * np.ones(len(y))]
    return np.asarray(X), np.asarray(y)

def generate_problem_matrix(domain_file_path, problem_folder_path, log_folder_path, output_path, title):
    """
    Generate the corresponding feature/label matrix from the given inputs
    Stores in the format of "title.npz" in the output_path
    Each npz file contain two attribute: "feature" and "label"
    
    Inputs
    ----------
    domain_file_path: the path to the domain.pddl
    problem_folder_path: the path to the problem_folder containing all the task problem.pddl for generating vectors
    plan_folder_path: the plan folder that contains all the log files corresponding to each problem task
    output_path: the place to store the generated problem matrix
    title: the name for the output npz file
    """
    # get the training problem names and initialise parameters
    problem_name_list = [f.split('.')[0] for f in os.listdir(problem_folder_path)]
    X = None
    Y = None
    problem_index = 0
    
    # generate the final vectors in X, Y
    for prob in problem_name_list:
        problem_path = problem_folder_path + "/" + prob + ".pddl"
        plan_path = log_folder_path + "/" + prob + "_1800.out"
        temp_X, temp_Y = generate_training_data(domain_file_path, problem_path, plan_path, problem_index)
        if X is None:
            X = temp_X
            Y = temp_Y
        elif temp_X is not None:
            X = np.vstack((X, temp_X))
            Y = np.vstack((Y, temp_Y))
        problem_index += 1
       
        
        
    # save the final training vectors
    np.savez(output_path+"/"+title+".npz", feature = X, label = Y)
        

In [13]:
X, y = generate_training_data("datasets/domain/transport/domain.pddl", 
                              "datasets/domain/transport/train/instance-1.pddl", 
                              "datasets/domain/transport/plans/instance-1_1800.out",
                              0)

In [15]:
X
y

array([[13.,  0.],
       [12.,  0.],
       [11.,  0.],
       [10.,  0.],
       [ 9.,  0.],
       [ 8.,  0.],
       [ 7.,  0.],
       [ 6.,  0.],
       [ 5.,  0.],
       [ 4.,  0.],
       [ 3.,  0.],
       [ 2.,  0.]])

# RankSVM

In [188]:
import itertools
import numpy as np

from sklearn import svm, linear_model
from sklearn.preprocessing import StandardScaler

def transform_pairwise(X, y):
    """
    Transforms data into pairs for convex relaxation of kendal rank correlation coef
    In this method, all pairs are choosen, except for those that have the same target value or equal cost
    Inputs
    ----------
    X : array, shape (n_samples, n_features)
        The input feature vec of states from of several problems
    y : array, shape (n_samples,) or (n_samples, 2)
        The input cost vector. If it's a 2D array, the second column represents
        the problem index
    Returns
    -------
    X_trans : array, shape (k, n_feaures)
        Difference between features of states (si - sj), only consider the state pair from the same problem
    y_trans : array, shape (k,)
        Output rank labels of values {-1, +1}, 1 represent si has potentially larger cost than sj (further away from goal)
    """
    X_new = []
    y_new = []
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            # skip if they have the same cost or are from different problem group
            continue
        # otherwise, make the new pair-wise data
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0])) # y = 1 if xi further away (larger cost), Vice Vesa
        # randomly output some negative values for training purpose
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new)


class RankSVM(svm.LinearSVC):
    """
    Performs pairwise ranking svm with an underlying LinearSVC model
    initialise with a C of regularization term
    default using hinge loss
    """
    
    def __init__(self, C = 1.0):
        super(RankSVM, self).__init__()
        self.C = C
        self.loss = 'hinge'
        self.fit_intercept = False
        self.max_iter = 9999
#         self.scaler = StandardScaler()
        
        
    def fit(self, X, y):
        """
        Fit a pairwise ranking model by first transfer it into pairwise than fitting
        Inputs
        ----------
        X : array, shape (n_samples, n_features)
        y : array, shape (n_samples,) or (n_samples, 2)
        Returns
        -------
        self
        """
        X_trans, y_trans = transform_pairwise(X, y)
        super(RankSVM, self).fit(X_trans, y_trans)
#         X_scaled = self.scaler.fit_transform(X_trans)
#         super(RankSVM, self).fit(X_scaled, y_trans)
        return self

    def predict(self, X):
        """
        Predict an ordering on X. For a list of n samples, this method
        returns a list from 0 to n-1 with the relative order of the rows of X.
        Inputs
        ----------
        X : array, shape (n_samples, n_features)
        Returns
        -------
        rtn: array, shape (n_samples,)
            Returns a list of integers representing the relative order of
            the rows in X.
        """
        if hasattr(self, 'coef_'):
#             return np.argsort(np.dot(self.scaler.transform(X), self.coef_.T).flatten())
            return np.argsort(np.dot(X, self.coef_.T).flatten())
        else:
            raise ValueError("Must call fit() prior to predict()")

    def score(self, X, y):
        """
        Returns the accuracy for the rank prediction, from 0-1
        """
        X_trans, y_trans = transform_pairwise(X, y)
        return np.mean(super(RankSVM, self).predict(X_trans) == y_trans)
    
    def h_val(self, x):
        """
        return pseduo heuristic for search
        """
        if hasattr(self, 'coef_'):
            x_r = x.reshape(1,-1)
#             return np.dot(self.scaler.transform(x_r), self.coef_.T)
            return np.dot(x_r, self.coef_.T)
        else:
            raise ValueError("Must call fit() prior to predict()")
        

# Search

In [189]:
class RankHeuristic(Heuristic):
    """
    Implement the heuristic using the trained RankSVM's coef for dot product
    
    Inadmissible, directly reflect the rank
    
    Default scale value is 10000
    """
    
    def __init__(self, svm : RankSVM, planning_graph, scale_val = 10000):
        super().__init__()
        self.svm = svm
        self.planning_graph = planning_graph
        self.scale = scale_val
        self.expand_nodes = 0
        
    def __call__(self, node):
        self.expand_nodes += 1
        if (self.planning_graph.task.goals <= node.state):
            print("reached goal state")
            return 0
        vec = generate_feature_vec_relaxed(self.planning_graph, node.state, 9999999999)
        h = round(self.svm.h_val(vec).item()*self.scale)
#         print(f'heristic values:{h}')
        return h

# Training

In [190]:
def test_optimal(domain_path, test_folder_path, log_folder_path, svm):
    """
    return: the percentage of problems that get solved and returned the optimal path
    if not optimal, print the difference of length as output
    """
    
    problem_name_list = [f.split('.')[0] for f in os.listdir(test_folder_path)]
    print(problem_name_list)
    correct_optimal = 0
    for prob in problem_name_list:
        # retrieve the problem and plan file path
        problem_path = test_folder_path + "/" + prob + ".pddl"
        plan_path = log_folder_path + "/" + prob + "_1800.out"

        
        # generate test plan and heuristic instance
        test_plan = RelaxedPlanningGraph(domain_path, problem_path)

        rank_h = RankHeuristic(svm, test_plan)
        
        # get solution using gbfs
        sol = gbfs(test_plan.task, rank_h)
        print(f'Nodes expanded:{rank_h.expand_nodes}')
        
        my_len = len(sol)
        true_len = len(read_plan(plan_path))
        print(f'For problem {prob}')
        if my_len != true_len:
            print(f'my plan length: {my_len}, optimal plan length: {true_len}')
        else:
            print(f'get true optimal length {my_len}')
            correct_optimal += 1
            
    return correct_optimal / len(problem_name_list)

In [203]:
# train blocks
generate_problem_matrix("datasets/domain/blocks/domain.pddl", 
                        "datasets/domain/blocks/train", 
                        "datasets/domain/blocks/plans", 
                        "datasets/results", 
                        "blocks-train")
svm_block = RankSVM(C = 0.1)
results = np.load("datasets/results/blocks-train.npz")
X = results['feature']
Y = results['label']

print(X.shape, Y.shape)

svm_block.fit(X, Y)

(94, 39) (94, 2)


E:\Apps\Anaconda\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RankSVM(C=0.1)

In [204]:
# test blocks 1
test_plan = RelaxedPlanningGraph("datasets/domain/blocks/domain.pddl", 
                                 "datasets/domain/blocks/blocks/blocks3/task01.pddl")

rank_h = RankHeuristic(svm_block, test_plan)

rtn = gbfs(test_plan.task, rank_h)
print(f'Nodes expanded:{rank_h.expand_nodes}')
print(f'plan length{len(rtn)}')
rtn

reached goal state
Nodes expanded:4
plan length2


[<Op (unstack b3 b2)>, <Op (put-down b3)>]

In [205]:
# test blocks 2
test_optimal("datasets/domain/blocks/domain.pddl",
            "datasets/domain/blocks/test-svm",
            "datasets/domain/blocks/plans",
            svm_block)

['blocks3-task04', 'blocks3-task05', 'blocks3-task06', 'blocks3-task07', 'blocks3-task08', 'blocks3-task09', 'blocks3-task10', 'blocks4-task05', 'blocks4-task06', 'blocks4-task07', 'blocks4-task08', 'blocks4-task09', 'blocks4-task10', 'blocks5-task04', 'blocks5-task05', 'blocks5-task06', 'blocks5-task07', 'blocks5-task08', 'blocks5-task09', 'blocks5-task10', 'blocks6-task06', 'blocks6-task07', 'blocks6-task08', 'blocks6-task09', 'blocks6-task10', 'blocks7-task02', 'blocks7-task03', 'blocks7-task04', 'blocks7-task05', 'blocks8-task02', 'blocks8-task03', 'blocks8-task04']
reached goal state
Nodes expanded:15
For problem blocks3-task04
get true optimal length 6
reached goal state
Nodes expanded:15
For problem blocks3-task05
get true optimal length 6
reached goal state
Nodes expanded:12
For problem blocks3-task06
get true optimal length 4
reached goal state
Nodes expanded:14
For problem blocks3-task07
get true optimal length 4
reached goal state
Nodes expanded:23
For problem blocks3-task08

0.53125

In [154]:
# train ferry
generate_problem_matrix("datasets/domain/ferry/domain.pddl", 
                        "datasets/domain/ferry/train", 
                        "datasets/domain/ferry/plans", 
                        "datasets/results", 
                        "ferry-train")

svm_ferry = RankSVM(C = 0.1)
results = np.load("datasets/results/ferry-train.npz")
X = results['feature']
Y = results['label']

print(X.shape, Y.shape)

svm_ferry.fit(X, Y)

(45, 24) (45, 2)


E:\Apps\Anaconda\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RankSVM(C=0.1)

In [155]:
# test ferry 1
test_plan = RelaxedPlanningGraph("datasets/domain/ferry/domain.pddl", 
                                 "datasets/domain/ferry/test/ferry-l2-c15.pddl")

rank_h = RankHeuristic(svm_ferry, test_plan)

rtn = gbfs(test_plan.task, rank_h)
print(f'Nodes expanded:{rank_h.expand_nodes}')
print(f'plan length{len(rtn)}')
rtn


reached goal state
Nodes expanded:376
plan length21


[<Op (board c14 l1)>,
 <Op (sail l1 l0)>,
 <Op (debark c14 l0)>,
 <Op (board c5 l0)>,
 <Op (sail l0 l1)>,
 <Op (debark c5 l1)>,
 <Op (board c6 l1)>,
 <Op (sail l1 l0)>,
 <Op (debark c6 l0)>,
 <Op (board c4 l0)>,
 <Op (sail l0 l1)>,
 <Op (debark c4 l1)>,
 <Op (board c3 l1)>,
 <Op (sail l1 l0)>,
 <Op (debark c3 l0)>,
 <Op (board c11 l0)>,
 <Op (sail l0 l1)>,
 <Op (debark c11 l1)>,
 <Op (board c1 l1)>,
 <Op (sail l1 l0)>,
 <Op (debark c1 l0)>]

In [156]:
# test ferry 2
test_optimal("datasets/domain/ferry/domain.pddl",
            "datasets/domain/ferry/test-svm",
            "datasets/domain/ferry/plans",
            svm_ferry)

['ferry-l2-c10', 'ferry-l2-c15', 'ferry-l2-c20', 'ferry-l2-c5', 'ferry-l3-c10', 'ferry-l3-c15', 'ferry-l3-c5', 'ferry-l4-c10', 'ferry-l4-c15', 'ferry-l4-c5', 'ferry-l5-c10', 'ferry-l5-c15', 'ferry-l5-c5']
reached goal state
Nodes expanded:38
For problem ferry-l2-c10
get true optimal length 10
reached goal state
Nodes expanded:376
For problem ferry-l2-c15
get true optimal length 21
reached goal state
Nodes expanded:1147
For problem ferry-l2-c20
my plan length: 28, optimal plan length: 25
reached goal state
Nodes expanded:17
For problem ferry-l2-c5
get true optimal length 6
reached goal state
Nodes expanded:118
For problem ferry-l3-c10
my plan length: 18, optimal plan length: 14
reached goal state
Nodes expanded:6649
For problem ferry-l3-c15
my plan length: 79, optimal plan length: 35
reached goal state
Nodes expanded:81
For problem ferry-l3-c5
my plan length: 17, optimal plan length: 15
reached goal state
Nodes expanded:524
For problem ferry-l4-c10
my plan length: 22, optimal plan lengt

0.38461538461538464

In [159]:
# train gripper
generate_problem_matrix("datasets/domain/gripper/domain.pddl", 
                        "datasets/domain/gripper/train", 
                        "datasets/domain/gripper/plans", 
                        "datasets/results", 
                        "gripper-train")
svm_gripper = RankSVM(C = 0.1)
results = np.load("datasets/results/gripper-train.npz")
X = results['feature']
Y = results['label']

print(X.shape, Y.shape)

svm_gripper.fit(X, Y)

(108, 24) (108, 2)


E:\Apps\Anaconda\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RankSVM(C=0.1)

In [160]:
# test gripper 1
test_plan = RelaxedPlanningGraph("datasets/domain/gripper/domain.pddl", 
                                 "datasets/domain/gripper/test/gripper-n4.pddl")

rank_h = RankHeuristic(svm_gripper, test_plan)

rtn = gbfs(test_plan.task, rank_h)
print(f'Nodes expanded:{rank_h.expand_nodes}')
print(f'plan length{len(rtn)}')
rtn

reached goal state
Nodes expanded:48
plan length11


[<Op (pick ball2 rooma left)>,
 <Op (pick ball3 rooma right)>,
 <Op (move rooma roomb)>,
 <Op (drop ball2 roomb left)>,
 <Op (drop ball3 roomb right)>,
 <Op (move roomb rooma)>,
 <Op (pick ball4 rooma left)>,
 <Op (pick ball1 rooma right)>,
 <Op (move rooma roomb)>,
 <Op (drop ball4 roomb left)>,
 <Op (drop ball1 roomb right)>]

In [162]:
# test gripper 2
test_optimal("datasets/domain/gripper/domain.pddl",
            "datasets/domain/gripper/test-svm",
            "datasets/domain/gripper/plans",
            svm_gripper)

['gripper-n13', 'gripper-n14', 'gripper-n15', 'gripper-n16', 'gripper-n17', 'gripper-n4', 'gripper-n5', 'gripper-n6', 'gripper-n7', 'gripper-n8', 'gripper-n9']
reached goal state
Nodes expanded:377
For problem gripper-n13
get true optimal length 39
reached goal state
Nodes expanded:396
For problem gripper-n14
get true optimal length 41
reached goal state
Nodes expanded:476
For problem gripper-n15
get true optimal length 45
reached goal state
Nodes expanded:516
For problem gripper-n16
get true optimal length 47
reached goal state
Nodes expanded:608
For problem gripper-n17
get true optimal length 51
reached goal state
Nodes expanded:48
For problem gripper-n4
get true optimal length 11
reached goal state
Nodes expanded:77
For problem gripper-n5
get true optimal length 15
reached goal state
Nodes expanded:96
For problem gripper-n6
get true optimal length 17
reached goal state
Nodes expanded:128
For problem gripper-n7
get true optimal length 21
reached goal state
Nodes expanded:150
For prob

1.0

In [177]:
# train zeno
generate_problem_matrix("datasets/domain/zenotravel/domain.pddl", 
                        "datasets/domain/zenotravel/train", 
                        "datasets/domain/zenotravel/plans", 
                        "datasets/results", 
                        "zenotravel-train")
svm_zeno = RankSVM(C = 0.1)
results = np.load("datasets/results/zenotravel-train.npz")
X = results['feature']
Y = results['label']

print(X.shape, Y.shape)

svm_zeno.fit(X, Y)

(120, 58) (120, 2)


E:\Apps\Anaconda\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RankSVM(C=0.1)

In [178]:
# test zeno 1
test_plan = RelaxedPlanningGraph("datasets/domain/zenotravel/domain.pddl", 
                                 "datasets/domain/zenotravel/test/zenotravel-cities2-planes2-people5-1564.pddl")

rank_h = RankHeuristic(svm_zeno, test_plan)

rtn = gbfs(test_plan.task, rank_h)
print(f'Nodes expanded:{rank_h.expand_nodes}')
print(f'plan length{len(rtn)}')
rtn

reached goal state
Nodes expanded:56
plan length6


[<Op (refuel plane2 city1 fl0 fl1)>,
 <Op (board person2 plane2 city1)>,
 <Op (refuel plane2 city1 fl1 fl2)>,
 <Op (fly plane2 city1 city0 fl2 fl1)>,
 <Op (debark person2 plane2 city0)>,
 <Op (fly plane2 city0 city1 fl1 fl0)>]

In [179]:
# test zeno 2
test_optimal("datasets/domain/zenotravel/domain.pddl",
            "datasets/domain/zenotravel/test-svm",
            "datasets/domain/zenotravel/plans",
            svm_zeno)

['zenotravel-cities2-planes3-people6-6510', 'zenotravel-cities2-planes3-people7-3468', 'zenotravel-cities2-planes4-people4-2248', 'zenotravel-cities2-planes4-people4-6874', 'zenotravel-cities2-planes4-people7-6599', 'zenotravel-cities2-planes5-people3-9854', 'zenotravel-cities3-planes2-people4-6913', 'zenotravel-cities3-planes2-people7-1956', 'zenotravel-cities3-planes3-people4-2981', 'zenotravel-cities4-planes4-people6-1235', 'zenotravel-cities4-planes4-people7-4853', 'zenotravel-cities4-planes5-people3-3894']
reached goal state
Nodes expanded:1191
For problem zenotravel-cities2-planes3-people6-6510
my plan length: 14, optimal plan length: 10
reached goal state
Nodes expanded:23566
For problem zenotravel-cities2-planes3-people7-3468
my plan length: 23, optimal plan length: 16


KeyboardInterrupt: 